In [1]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder
import helper_funcs
import torch
import numpy as np
import faiss
import threading

KeyboardInterrupt: 

In [ ]:
encoder = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
#cross_encoder = CrossEncoder('cross-encoder/stsb-distilroberta-base')

In [ ]:
d = 384
description = "Flat"
#description = "IVF1024(PQ16x4),PQ16"
# description = "HNSW32_SQ8"
# description = "HNSW32_PQ4"
# description = "PCA64,IVF32(PQ2x4),Flat"
# description = "OPQ16,PCA64,IVF32(PQ2x4),Flat"
# description = "IVF64(PQ4x6),Flat"
# description = "IVF64(SQ8),Flat"
# description = "IVF64(PQ6x6),Flat"

index = helper_funcs.create_index(d, description, gpu=True)

print(index.ntotal)

0


In [ ]:
files = []
for i in range(1, 10):
    f = torch.load(f'../msmarco-vec/msmarco-vectors-{i}.pt')
    files.append(f)

xb = np.concatenate(files)

In [ ]:
print(xb.shape)

In [ ]:
def populate_index(xb, index):
    faiss.normalize_L2(xb)

    index.train(xb)
    index.add(xb)

In [ ]:
populate_index(xb, index)

print(index.ntotal)

In [ ]:
phase = "dev.small"

collection = helper_funcs.tsv_to_df("../collectionandqueries/collection", col_name=["id", "paragraph"], index_col="id")
queries = helper_funcs.tsv_to_df(f"../collectionandqueries/queries.{phase}", col_name=["query number", "query text"], index_col="query number")
qrels = helper_funcs.tsv_to_df(f"../collectionandqueries/qrels.{phase}", col_name=["query id", "del.", "document id", "del.2"], index_col="query id").drop(labels=["del.", "del.2"], axis=1)

In [ ]:
total_mrr = []

def search_thread(query, document):
    query_val = query.iloc[0]["query text"]
    document_val = document.iloc[0]["paragraph"]
    #reranked_results = helper_funcs.search_and_rerank(query_val, 10, encoder=encoder, cross_encoder=cross_encoder, index=index, collection=collection)
    reranked_results = helper_funcs.search(query_val, 10, encoder=encoder, index=index)

    result = [reranked_results[i][0][0] for i in range(len(reranked_results))]
    result = list(result)
    matching_doc = document.iloc[0].name

    mrr = helper_funcs.mrr_at(10, result, matching_doc)
    total_mrr.append(mrr)

In [ ]:
# Calculating MRR@10
generator = helper_funcs.query_document_generator(collection, queries, qrels)

for query, document in generator:
    t = threading.Thread(target=search_thread, args=[query, document])
    t.run()
    print()

print(f"MRR@10 {sum(total_mrr) / len(total_mrr)}")

In [ ]:
total_mrr